# Master course in Object Recognition
## Practice 1

### Title: Deep learning advanced architectures

The goal is to practice advanced deep learning architectures for multi-label classification in [Pascal VOC dataset](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/index.html). We specifically check ResNet50, Inception and MobileNet. We will see 1) how pretrained ResNet50 on imagenet performs on multi-label images, 2) how to modify classification head and 3) implementation of F1 metric.

### NOTES

- Hyperparameters are modifiable,
- The dataset is PASCAL VOC 2012,
- The code uses the KERAS library,
- The code can run in google colab.
- How to finetune on a pretrained model not included (i.e. freeze the pretrained network and train the head, then finetune everything),
- No validation set has been defined. The test and validation sets are the same.

In [1]:
# Imports

from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from keras.layers import (
    Dense,
    GlobalAveragePooling2D,
)
import tensorflow.keras.applications as app
import pandas as pd
import csv
import time

2025-03-07 05:41:28.987335: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741326089.005416  613451 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741326089.011045  613451 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-07 05:41:29.029933: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Importing from .py files

from metrics import f1_metric, mean_average_precision, subset_accuracy_metric
from config import *
from experiment_config import experiments
from train_and_test import train_and_test
from load_data import load_data, create_dataset

In [3]:
train_list = load_data(TRAIN_TXT)
test_list = load_data(TEST_TXT)

# Create dictionaries to store datasets for different batch sizes
train_datasets = {}
test_datasets = {}

start_time = time.time()
# Iterate over batch sizes and create datasets
for batch_size in BATCH_SIZES:
    train_datasets[batch_size] = create_dataset(
        train_list, batch_size, is_training=True
    )
    test_datasets[batch_size] = create_dataset(test_list, batch_size, is_training=False)
print(f"Time taken to create datasets: {time.time() - start_time} seconds")

I0000 00:00:1741326092.568221  613451 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10147 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:03:00.0, compute capability: 8.6


Time taken to create datasets: 1.8166263103485107 seconds


In [4]:
# Run model experiments
exp_name = "model-experiments"
for exp in experiments[exp_name][7:]:
    print(f"Defining model: {exp.title}")

    # Select the corresponding network class
    mynet = getattr(getattr(app, exp.net_name[0]), exp.net_name[1])

    # Create the base pre-trained model
    base_model = (
        mynet(include_top=False)
        if exp.train_from_scratch
        else mynet(weights="imagenet", include_top=False)
    )

    # Add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)  # Fully connected layer
    predictions = Dense(num_classes, activation=exp.last_layer_activation)(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model, exp_name, exp, train_dataset, test_dataset, train_list, test_list
    )

Defining model: mobilenet_v2 pretraining no-warmup


/tmp/ipykernel_613451/3644587405.py:13: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  else mynet(weights="imagenet", include_top=False)


In training loop: mobilenet_v2 pretraining no-warmup


I0000 00:00:1741326108.741813  613451 service.cc:148] XLA service 0x200f21a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741326108.741865  613451 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti, Compute Capability 8.6
2025-03-07 05:41:49.108484: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1741326110.602709  613451 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-07 05:41:52.645342: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4101', 16 bytes spill stores, 16 bytes spill loads

2025-03-07 05:41:53.420367: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4101_0', 104 byt

Time taken for training one epoch: 54.63s
Epoch 0 training loss: 0.14, acc: 0.92, f1: 0.63, mAP: 0.78
Time taken for testing one epoch: 22.80s
Epoch 0 test loss: 0.55, acc: 0.74, f1: 0.48, mAP: 0.65
Time taken for training one epoch: 44.72s
Epoch 1 training loss: 0.09, acc: 0.97, f1: 0.74, mAP: 0.88
Time taken for testing one epoch: 22.47s
Epoch 1 test loss: 0.45, acc: 0.77, f1: 0.50, mAP: 0.66
Time taken for training one epoch: 44.62s
Epoch 2 training loss: 0.07, acc: 0.98, f1: 0.80, mAP: 0.92
Time taken for testing one epoch: 22.65s
Epoch 2 test loss: 0.45, acc: 0.73, f1: 0.24, mAP: 0.55
Time taken for training one epoch: 44.71s
Epoch 3 training loss: 0.06, acc: 0.99, f1: 0.83, mAP: 0.94
Time taken for testing one epoch: 22.61s
Epoch 3 test loss: 0.27, acc: 0.82, f1: 0.51, mAP: 0.67
Time taken for training one epoch: 44.86s
Epoch 4 training loss: 0.05, acc: 0.99, f1: 0.86, mAP: 0.95
Time taken for testing one epoch: 22.54s
Epoch 4 test loss: 0.38, acc: 0.79, f1: 0.53, mAP: 0.65
Time 

In [5]:
# Determine the best experiment of the 9 model experiments

df = pd.read_csv(RESULTS_DIR / f"model-experiments.csv")
best_id = df.loc[df["Test mAP"].idxmax(), "ID"]

best_model_experiment_config = next(
    exp for exp in experiments["model-experiments"] if exp.id == best_id
)

best_model_experiment_config

ExperimentConfig(id=2, title='resnet50 pretraining warmup', net_name=['resnet50', 'ResNet50'], train_from_scratch=False, warm_up=True, batch_size=32, n_epochs=12, last_layer_activation='sigmoid', learning_rate=0.001, loss='binary_crossentropy')

: 

In [ ]:
# Run hyperparameter experiments

exp_name = "hyperparameter-experiments"
for exp in experiments[exp_name]:
    print(f"Defining model: {exp.title}")
    print(
        f"Reusing net_name: {best_model_experiment_config.net_name}, train_from_scratch: {best_model_experiment_config.train_from_scratch}, warm_up: {best_model_experiment_config.warm_up} from best model experiment"
    )
    exp.net_name = best_model_experiment_config.net_name
    exp.train_from_scratch = best_model_experiment_config.train_from_scratch
    exp.warm_up = best_model_experiment_config.warm_up

    # Select the corresponding network class
    mynet = getattr(getattr(app, exp.net_name[0]), exp.net_name[1])

    # Create the base pre-trained model
    base_model = (
        mynet(include_top=False)
        if exp.train_from_scratch
        else mynet(weights="imagenet", include_top=False)
    )

    # Add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)  # Fully connected layer
    predictions = Dense(num_classes, activation=exp.last_layer_activation)(x)

      # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model, exp_name, exp, train_dataset, test_dataset, train_list, test_list
    )

Defining model: batch_size: 16, learning_rate: 0.001
Reusing net_name: ['resnet50', 'ResNet50'], train_from_scratch: False, warm_up: True from best model experiment
In training loop: batch_size: 16, learning_rate: 0.001
Time taken for training one epoch: 78.57s
Epoch 0 training loss: 0.18, acc: 0.87, f1: 0.51, mAP: 0.70
Time taken for testing one epoch: 39.64s
Epoch 0 test loss: 0.19, acc: 0.87, f1: 0.36, mAP: 0.62
Time taken for training one epoch: 68.56s
Epoch 1 training loss: 0.11, acc: 0.95, f1: 0.67, mAP: 0.83
Time taken for testing one epoch: 39.49s
Epoch 1 test loss: 0.17, acc: 0.90, f1: 0.57, mAP: 0.74
Unfreezing base model at epoch 2
Time taken for training one epoch: 68.15s
Epoch 2 training loss: 0.08, acc: 0.97, f1: 0.76, mAP: 0.89
Time taken for testing one epoch: 39.61s
Epoch 2 test loss: 0.21, acc: 0.88, f1: 0.52, mAP: 0.71
Time taken for training one epoch: 70.75s
Epoch 3 training loss: 0.07, acc: 0.98, f1: 0.81, mAP: 0.92
Time taken for testing one epoch: 39.77s
Epoch 3

2025-03-07 07:14:51.522661: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6302_0', 112 bytes spill stores, 224 bytes spill loads

2025-03-07 07:14:51.647094: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6785', 104 bytes spill stores, 104 bytes spill loads

2025-03-07 07:14:51.719335: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11157', 24 bytes spill stores, 24 bytes spill loads

2025-03-07 07:14:51.776295: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11157', 24 bytes spill stores, 28 bytes spill loads

2025-03-07 07:14:51.907243: I external/local_xla/xla

Time taken for training one epoch: 78.97s
Epoch 0 training loss: 0.17, acc: 0.89, f1: 0.56, mAP: 0.74
Time taken for testing one epoch: 25.08s
Epoch 0 test loss: 0.24, acc: 0.74, f1: 0.04, mAP: 0.44
Time taken for training one epoch: 67.70s
Epoch 1 training loss: 0.09, acc: 0.97, f1: 0.74, mAP: 0.88
Time taken for testing one epoch: 25.03s
Epoch 1 test loss: 0.18, acc: 0.90, f1: 0.57, mAP: 0.72
Unfreezing base model at epoch 2
Time taken for training one epoch: 67.36s
Epoch 2 training loss: 0.07, acc: 0.98, f1: 0.81, mAP: 0.93
Time taken for testing one epoch: 25.02s
Epoch 2 test loss: 0.40, acc: 0.73, f1: 0.21, mAP: 0.48
Time taken for training one epoch: 67.44s
Epoch 3 training loss: 0.05, acc: 0.99, f1: 0.86, mAP: 0.95
Time taken for testing one epoch: 24.48s
Epoch 3 test loss: 0.24, acc: 0.86, f1: 0.53, mAP: 0.70
Time taken for training one epoch: 67.61s
Epoch 4 training loss: 0.04, acc: 0.99, f1: 0.89, mAP: 0.97
Time taken for testing one epoch: 25.05s
Epoch 4 test loss: 0.26, acc

2025-03-07 08:11:49.450768: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6300_0', 204 bytes spill stores, 204 bytes spill loads

2025-03-07 08:11:50.006851: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6300', 192 bytes spill stores, 512 bytes spill loads

2025-03-07 08:11:50.846266: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6791', 104 bytes spill stores, 104 bytes spill loads

2025-03-07 08:11:51.758404: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11163', 496 bytes spill stores, 496 bytes spill loads

2025-03-07 08:11:52.012772: I external/local_xla/

Time taken for training one epoch: 86.91s
Epoch 0 training loss: 0.18, acc: 0.88, f1: 0.58, mAP: 0.74
Time taken for testing one epoch: 18.99s
Epoch 0 test loss: 0.28, acc: 0.70, f1: 0.00, mAP: 0.53


In [ ]:
# Determine the best experiment of the 9 hyperparameter experiments

df = pd.read_csv(RESULTS_DIR / f"hyperparameter-experiments.csv")
best_id = df.loc[df["Test mAP"].idxmax(), "ID"]

best_hyperparameter_experiment_config = next(
    exp for exp in experiments["hyperparameter-experiments"] if exp.id == best_id
)

best_hyperparameter_experiment_config

In [ ]:
# Run augmentation experiments

exp_name = "augmentation-experiments"
for exp in experiments[exp_name]:
    print(f"Defining model: {exp.title}")
    print(
        f"Reusing net_name: {best_hyperparameter_experiment_config.net_name}, train_from_scratch: {best_hyperparameter_experiment_config.train_from_scratch}, warm_up: {best_hyperparameter_experiment_config.warm_up} from best hyperparameter experiment"
    )
    exp.net_name = best_hyperparameter_experiment_config.net_name
    exp.train_from_scratch = best_hyperparameter_experiment_config.train_from_scratch
    exp.warm_up = best_hyperparameter_experiment_config.warm_up
    exp.batch_size = best_hyperparameter_experiment_config.batch_size
    exp.learning_rate = best_hyperparameter_experiment_config.learning_rate
    exp.loss = best_hyperparameter_experiment_config.loss
    exp.last_layer_activation = (
        best_hyperparameter_experiment_config.last_layer_activation
    )

    # Select the corresponding network class
    mynet = getattr(getattr(app, exp.net_name[0]), exp.net_name[1])

    # Create the base pre-trained model
    base_model = (
        mynet(include_top=False)
        if exp.train_from_scratch
        else mynet(weights="imagenet", include_top=False)
    )

    # Add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)  # Fully connected layer
    predictions = Dense(num_classes, activation=exp.last_layer_activation)(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model, exp_name, exp, train_dataset, test_dataset, train_list, test_list
    )

In [ ]:
# Run classifier head experiments

exp_name = "classfier_head_experiments"
for exp in experiments[exp_name]:
    print(f"Defining model: {exp.title}")
    print(
        f"Reusing net_name: {best_hyperparameter_experiment_config.net_name}, train_from_scratch: {best_hyperparameter_experiment_config.train_from_scratch}, warm_up: {best_hyperparameter_experiment_config.warm_up} from best hyperparameter experiment"
    )
    exp.net_name = best_hyperparameter_experiment_config.net_name
    exp.train_from_scratch = best_hyperparameter_experiment_config.train_from_scratch
    exp.warm_up = best_hyperparameter_experiment_config.warm_up
    exp.batch_size = best_hyperparameter_experiment_config.batch_size
    exp.learning_rate = best_hyperparameter_experiment_config.learning_rate
    exp.loss = best_hyperparameter_experiment_config.loss
    exp.last_layer_activation = (
        best_hyperparameter_experiment_config.last_layer_activation
    )

    # Select the corresponding network class
    mynet = getattr(getattr(app, exp.net_name[0]), exp.net_name[1])

    # Create the base pre-trained model
    base_model = (
        mynet(include_top=False)
        if exp.train_from_scratch
        else mynet(weights="imagenet", include_top=False)
    )

    # Add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)  # Fully connected layer
    predictions = Dense(num_classes, activation=exp.last_layer_activation)(x)

        # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model, exp_name, exp, train_dataset, test_dataset, train_list, test_list
    )